In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

In [ ]:
path = r'/content/training_data'                     # use your path
all_files = glob.glob(path + "/*.csv")
labels = pd.read_csv('/content/CIS-PD_Training_Data_IDs_Labels.csv', index_col=None, header=0)

In [ ]:
X = []
for filename in all_files:
  values = []
  data = pd.read_csv(filename, index_col=None, header=0, nrows = 5803)
  values.append(data['X'].values)
  values.append(data['Y'].values)
  values.append(data['Z'].values)
  values = np.array(values)
  X.append(values)

X = np.stack(X)

y = []
for filename in all_files:
  head, tail = os.path.split(filename)
  for index, row in labels.iterrows():
    measurement_id = row['measurement_id'] + ".csv"
    if (measurement_id == tail):
      y.append(row['on_off'])
      break
y = np.array(y)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 15, 1
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  #_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  #return accuracy

In [ ]:
print (evaluate_model(X_train, X_test, y_train, y_test))

In [ ]:
scores = list()
for r in range(10):
  score = evaluate_model(X_train, X_test, y_train, y_test)
  score = score * 100.0
  print('>#%d: %.3f' % (r+1, score))
  scores.append(score)
print(scores)
m, s = np.mean(scores), np.std(scores)
print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))